Matrix 1 (control), sensor 0 data is corrupt. Copy Mat 1 S 1 to Mat 1 S 0.

In [7]:
import pickle
import numpy as np
import pandas as pd

In [8]:
with open("raw_sensor_data.pkl", "rb") as f:
    sensor_data = pickle.load(f)

In [9]:
def build_comparison_df(sensor_data, mat, sensor):
    data_lengths = []
    for heater_idx in sensor_data[f"mat_{mat}"][sensor]:
        heater_data_len = sensor_data[f"mat_{
            mat}"][sensor][heater_idx].shape[0]
        data_lengths.append(heater_data_len)

    min_len = min(data_lengths)
    heater_dict = {}
    for heater_idx in sensor_data[f"mat_{mat}"][sensor]:
        heater_dates = sensor_data[f"mat_{
            mat}"][sensor][heater_idx]["Date"].to_numpy()
        heater_data_len = len(heater_dates)
        if heater_data_len > min_len:
            heater_dates = heater_dates[:min_len]
        heater_dict[heater_idx] = heater_dates

    for heater_idx in range(1, 10):
        diff_seconds = (heater_dict[0] - heater_dict[heater_idx]
                        ).astype("timedelta64[s]").astype(np.int32)
        heater_dict[f"d 0-{heater_idx}"] = diff_seconds

    df = pd.DataFrame(heater_dict)
    return df

In [10]:
def impute_data_mean(df, after):
    df_copy = df.copy()
    line = df_copy.iloc[after:after+2].mean()

    line["Sensor Index"] = int(line["Sensor Index"])
    line["Sensor ID"] = int(line["Sensor ID"])
    line["Time Since PowerOn"] = int(line["Time Since PowerOn"])
    line["Real time clock"] = int(line["Real time clock"])
    line["Heater Profile Step Index"] = int(line["Heater Profile Step Index"])
    line["Scanning Mode Enabled"] = int(line["Scanning Mode Enabled"])
    line["Scanning Cycle Index"] = int(line["Scanning Cycle Index"])
    line["Label Tag"] = int(line["Label Tag"])
    line["Error Code"] = int(line["Error Code"])
    line["Date"] = line["Date"].floor("S")

    index = df_copy.iloc[after:after+2].index
    index = int((index[0] + index[1]) / 2)
    df_copy.loc[index] = line
    df_copy.sort_index(inplace=True)
    return df_copy

In [31]:
MATRIX = 1

In [76]:
for i in range(7, 8):
    df = build_comparison_df(sensor_data, MATRIX, i)
    df.to_excel(f"mat_{MATRIX}_s{i}_diff.xlsx")

In [68]:
sensor_data[f"mat_{MATRIX}"][6][0].iloc[3263:3268]

,Sensor Index,Sensor ID,Time Since PowerOn,Real time clock,Temperature,Pressure,Relative Humidity,Resistance Gassensor,Heater Profile Step Index,Scanning Mode Enabled,Scanning Cycle Index,Label Tag,Error Code,Date
334779,6,355877689,80298875,1740660913,35.677269,926.281128,10.510284,102400000.0,0,1,1,0,0,2025-02-27 15:55:13
334878,6,355877689,80323593,1740660938,35.669781,926.259094,10.475834,102400000.0,0,1,1,0,0,2025-02-27 15:55:38
335069,6,355877689,80372791,1740660987,35.335419,926.249146,10.600574,102400000.0,0,1,1,0,0,2025-02-27 15:56:27
335168,6,355877689,80397378,1740661012,35.033497,926.253357,10.672688,102400000.0,0,1,1,0,0,2025-02-27 15:56:52
335264,6,355877689,80422093,1740661037,34.741554,926.291870,10.749983,102400000.0,0,1,1,0,0,2025-02-27 15:57:17


In [75]:
sensor = 7
heater = 1
after = 2765
df_copy = impute_data_mean(sensor_data[f"mat_{MATRIX}"][sensor][heater], after)
sensor_data[f"mat_{MATRIX}"][sensor][heater] = df_copy.copy()
df_copy.iloc[after-1:after+3]

,Sensor Index,Sensor ID,Time Since PowerOn,Real time clock,Temperature,Pressure,Relative Humidity,Resistance Gassensor,Heater Profile Step Index,Scanning Mode Enabled,Scanning Cycle Index,Label Tag,Error Code,Date
285833,7,355868473,67869307,1740648482,34.700222,927.836243,10.338248,2513654.500,1,1,1,0,0,2025-02-27 12:28:02
285932,7,355868473,67893899,1740648507,34.568222,927.812927,10.448643,2509035.250,1,1,1,0,0,2025-02-27 12:28:27
286026,7,355868473,67918550,1740648531,34.535843,927.804535,10.531455,2520669.125,1,1,1,0,0,2025-02-27 12:28:51
286121,7,355868473,67943202,1740648556,34.503464,927.796143,10.614268,2532303.000,1,1,1,0,0,2025-02-27 12:29:16


In [37]:
sensor_data[f"mat_{MATRIX}"][0] = sensor_data[f"mat_{MATRIX}"][1].copy()

In [77]:
with open("imputed_sensor_data.pkl", "wb") as f:
    pickle.dump(sensor_data, f)